<a href="https://colab.research.google.com/github/qamtam/Hands-on-machine-learning/blob/main/CH14_SET10_with_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
import numpy as np
import pandas as pd
from functools import partial

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.
X_test = X_test/255.
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]


11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
X_test.shape
#

(10000, 28, 28)

In [ ]:

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]


In [ ]:
X_test.shape
#to make a color layer "visible" to convolution layers

(10000, 28, 28, 1)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
from keras.layers import MaxPool2D, Flatten, Dropout, Conv2D, Dense, Input
model = keras.models.Sequential([
                                 Input(shape=(28,28,1)),
                               # Normalization,
                                 Conv2D(32, 5, padding="same", activation="tanh"),
                                 Conv2D(64, 3,  padding="same", activation="tanh"),
                                 MaxPool2D(),
                                 Conv2D(128,2,  activation="tanh", padding="same"),
                                 Conv2D(128,2,  activation="tanh", padding="same"),
                                 MaxPool2D(),
                                 Conv2D(256,2,  activation="relu", padding="same"),
                                 MaxPool2D(),
                                 Flatten(),
                                 Dense(128, activation="selu"),
                                 Dropout(0.2),
                                 Dense(64, activation="selu"),
                                 Dropout(0.4),
                                 Dense(10, activation="softmax")
])

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
model = keras.models.Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters = 256, kernel_size = 2,padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 256, kernel_size = 2,padding = 'Same', 
                 activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
ddddddsadas


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

model.compile(optimizer="nadam", metrics=["accuracy"], loss="sparse_categorical_crossentropy")
model.fit_generator(datagen.flow(X_train, y_train), epochs=30, validation_data=(X_valid, y_valid), callbacks=[learning_rate_reduction])
model.evaluate(X_test, y_test)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
1719/1719 [==============================] - 28s 11ms/step - loss: 0.5888 - accuracy: 0.8046 - val_loss: 0.0417 - val_accuracy: 0.9862
Epoch 2/30
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0921 - accuracy: 0.9730 - val_loss: 0.0543 - val_accuracy: 0.9834
Epoch 3/30
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0720 - accuracy: 0.9792 - val_loss: 0.0329 - val_accuracy: 0.9910
Epoch 4/30
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0681 - accuracy: 0.9803 - val_loss: 0.0302 - val_accuracy: 0.9932
Epoch 5/30
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0575 - accuracy: 0.9837 - val_loss: 0.0338 - val_accuracy: 0.9908
Epoch 6/30
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0590 - accuracy: 0.9835 - val_loss: 0.0365 - val_accuracy: 0.9912
Epoch 7/30
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0511 - accuracy: 0.9851 - val_loss: 0.0262 -

[0.029239634051918983, 0.9912999868392944]

In [ ]:
fr
Normalization = keras.layers.experimental.preprocessing.Normalization()
# normalization axis 1,2 wplywa katastrofalnie, przy tym każdy pixel w batchu ma swoją normalizację
#normalization axis 3 powoduje, że wszystkie batche łącznie mają mu = 0 stdev =  1 
Normalization.adapt(X_valid)
#print(Normalization.mean)
X_v = Normalization(X_valid)
print("@@#@#")
print(np.mean(np.std(X_v, axis=0), axis=(0,1,2)))
print(np.mean(X_valid, axis=(0,1,2)))


NameError: ignored

In [ ]:

"""
model = keras.models.Sequential([
                                 Normalization(input),
                                 Conv2D(32, 5, stride=1, padding="same", activation="tanh"),
                                 Conv2D(64, 3, stride=1, padding="same", activation="tanh"),
                                 MaxPool2D(),
                                 Conv2D(128,2, stride=1, activation="tanh", padding="same"),
                                 Conv2D(128,2, stride=1, activation="tanh", padding="same"),
                                 MaxPool2D(),
                                 Flatten(),
                                 Dense(128, activation="selu"),
                                 Dropout(0.2),
                                 Dense(64, activation="selu"),
                                 Dropout(0.4),
                                 Dense(10, activation="softmax")
])

"""

In [ ]:
import sys
import numpy as np
np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=2000)
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]


X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]
#print(X_train[0])
#print(X_valid[1])
Normalization = keras.layers.experimental.preprocessing.Normalization(axis=(2,3))
#(1,2) to po każdej komórce
X_valid = X_valid[:2]
print(X_valid[:,4,:])

print(np.mean(X_valid, axis=(0,1)))


Normalization.adapt(X_valid)

print(Normalization(X_test[:2]))





In [ ]:

input_data = np.array([[-1.41], [-0.70], [0.]], np.float32)
print(np.mean(input_data))

In [ ]:
print(X_valid)

In [ ]:
series = generate_time_series(10000, n_steps + 10)
X_train, y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -10, 0]
X_test, y_test = series[9000:, :n_steps], series[9000:, -10, 0]


model = keras.models.Sequential([
                                 keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
                                 keras.layers.SimpleRNN(20),
                                 keras.layers.Dense(10)
])
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20)

X_new = series[:, :n_steps]
Y_pred = model.predict(X_new)